# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [25]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [26]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [27]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f74e1f5ea90>)

In [28]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [29]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [30]:
# # For testing, restrict training data to that before a hypothetical predictor submission date
# HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
# df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [31]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [32]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [33]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [34]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [35]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [36]:
temp = pd.read_csv('temperature_data.csv')
temp['date_st'] = temp['Date'].apply(lambda e: e[5:])
temp['id'] = temp['GeoID'] + '_' + temp['date_st']
id_temp = dict(zip( temp['id'], temp['temp'] ))
id_holiday = dict(zip( temp['id'], temp['Holiday'] ))
tf = temp[['date_st','temp']]
tf = tf.groupby(['date_st']).mean().reset_index()
date_temp_avg = dict(zip( tf['date_st'], tf['temp'] ))
tf = temp[['date_st','Holiday']]
tf = tf.groupby(['date_st'])['Holiday'].agg(pd.Series.mode).reset_index()
date_holiday_avg = dict(zip( tf['date_st'], tf['Holiday'] ))
id_temp

{'Afghanistan__nan_01-01': 2.7268292682926836,
 'Afghanistan__nan_01-02': 2.9152032520325206,
 'Afghanistan__nan_01-03': 2.9459349593495934,
 'Afghanistan__nan_01-04': 2.7965853658536584,
 'Afghanistan__nan_01-05': 2.4123577235772347,
 'Afghanistan__nan_01-06': 2.3713008130081303,
 'Afghanistan__nan_01-07': 2.4534959349593493,
 'Afghanistan__nan_01-08': 2.2549593495934954,
 'Afghanistan__nan_01-09': 2.219268292682927,
 'Afghanistan__nan_01-10': 2.2303252032520327,
 'Afghanistan__nan_01-11': 2.563983739837398,
 'Afghanistan__nan_01-12': 2.7221951219512204,
 'Afghanistan__nan_01-13': 2.389186991869919,
 'Afghanistan__nan_01-14': 2.290813008130081,
 'Afghanistan__nan_01-15': 2.217967479674797,
 'Afghanistan__nan_01-16': 2.2412195121951224,
 'Afghanistan__nan_01-17': 2.432926829268292,
 'Afghanistan__nan_01-18': 2.406747967479675,
 'Afghanistan__nan_01-19': 2.1417886178861787,
 'Afghanistan__nan_01-20': 2.324878048780488,
 'Afghanistan__nan_01-21': 2.0624390243902444,
 'Afghanistan__nan_01

In [37]:
# Set number of past days to use to make predictions
nb_lookback_days = 30
date_ls = []
geoid_ls = []
country_ls = []
newcase_ls = []
# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
train_geo_ids = [e for e in geo_ids]
geoid_arr = np.zeros(len(train_geo_ids)+1)
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]
        
        date_ls += [ list(gdf['Date'])[d] ]
        geoid_ls += [ list(gdf['GeoID'])[d] ]
        country_ls += [ list(gdf['CountryName'])[d]  ] 
        newcase_ls += [ list(gdf['NewCases'])[d]  ] 
        
        date_st = str(date_ls[-1])[5:10] 
        id_ = geoid_ls[-1] + '_' + date_st

        temperature = date_temp_avg[date_st]
        holiday = date_holiday_avg[date_st]
        if id_ in id_temp:
            temperature = id_temp[id_]
            holiday = id_holiday[id_]            
        
        # Flatten all input data so it fits Lasso input format.
        geoid_arr = np.zeros(len(train_geo_ids)+1)
        geoid_arr[ train_geo_ids.index(g) ] = 1
        X_sample = np.concatenate([geoid_arr, [temperature,holiday], X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d]
        X_samples.append(X_sample)
        y_samples.append(y_sample)


X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()
with open('train_geo_ids.txt', 'w') as f:
    f.write('\n'.join(train_geo_ids))
    
print(X_samples.shape)

(85440, 660)


In [38]:
import datetime
train_idx = [i for i in range(len(date_ls)) if date_ls[i] <= datetime.date(2020,7,31) ]
test_idx = [i for i in range(len(date_ls)) if date_ls[i] >= datetime.date(2020,8,1) ]
train_idx = np.array(train_idx)
test_idx = np.array(test_idx)

In [39]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [40]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = X_samples[train_idx,:], X_samples[test_idx,:],y_samples[train_idx], y_samples[test_idx]
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape,)

(48861, 660) (36579, 660) (48861,) (36579,)


In [41]:
!pip install lightgbm

In [42]:
import random
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(42) 

In [43]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.

lasso_model = Lasso(random_state=42)
lasso_model.fit(X_train, y_train)

print('Lasso result:')
# Evaluate model
train_preds = lasso_model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))
test_preds = lasso_model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))
with open('models/model_lasso.pkl', 'wb') as model_file:
    pickle.dump(lasso_model, model_file)

from lightgbm import LGBMRegressor
lgbm_model = LGBMRegressor(random_state=42)
lgbm_model.fit(X_train, y_train)
# Evaluate model
train_preds = lgbm_model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('lgbm result:')
print('Train MAE:', mae(train_preds, y_train))
test_preds = lgbm_model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))
with open('models/model_lgbm.pkl', 'wb') as model_file:
    pickle.dump(lgbm_model, model_file)

import xgboost as xgb
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)
# Evaluate model
train_preds = xgb_model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('xgb result:')
print('Train MAE:', mae(train_preds, y_train))
test_preds = xgb_model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))
with open('models/model_xgb.pkl', 'wb') as model_file:
    pickle.dump(xgb_model, model_file)


Lasso result:
Train MAE: 118.72727991825965
Test MAE: 447.0125097404181
lgbm result:
Train MAE: 84.90123948303615
Test MAE: 636.8706391310907
xgb result:
Train MAE: 52.951500424283076
Test MAE: 624.6861186879314


In [44]:
#with geoid
# Lasso result:
# Train MAE: 118.69420734991954
# Test MAE: 447.06846062728135
# lgbm result:
# Train MAE: 85.51717187475573
# Test MAE: 650.7371662424473
# xgb result:
# Train MAE: 54.2057830354577
# Test MAE: 633.3338386238983

In [45]:
# None geoid result:
# Lasso result:
# Train MAE: 119.2762230373797
# Test MAE: 469.3599710917388
# lgbm result:
# Train MAE: 85.58646245869078
# Test MAE: 665.2549123382636
# xgb result:
# Train MAE: 51.797418546131624
# Test MAE: 650.3725903829637

In [46]:
# Save the best model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(lasso_model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [47]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [48]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0
2020-08-05: 0
2020-08-06: 7.3965300629590915
2020-08-07: 10.650240165017394
2020-08-08: 7.500342299602266
2020-08-09: 3.3086164764963346
2020-08-10: 3.5415513216760584
2020-08-11: 0
2020-08-12: 0
2020-08-13: 5.139528777691787
2020-08-14: 1.802766263031657
2020-08-15: 0
2020-08-16: 0
2020-08-17: 0
2020-08-18: 0
2020-08-19: 0
2020-08-20: 0
2020-08-21: 0.3595418268590942
2020-08-22: 0
2020-08-23: 6.329626078578531
2020-08-24: 14.454134376543239
2020-08-25: 6.853029041836848
2020-08-26: 10.243713649480313
2020-08-27: 17.353831351393836
2020-08-28: 15.128847381115335
2020-08-29: 9.250552786050742
2020-08-30: 14.473732459851668
2020-08-31: 31.46883191540568

Predicting for Afghanistan__nan
2020-08-01: 92.87378979947317
2020-08-02: 1.2652484666498758
2020-08-03: 72.53020814220258
2020-08-04: 134.0990297639328
2020-08-05: 127.21088262915066
2020-08-06: 95.64459781506174
2020-08-07: 14.327124661152077
2020-08-08:

2020-08-17: 97.2497800880091
2020-08-18: 105.54466940877651
2020-08-19: 59.143306633523686
2020-08-20: 10.424398871410167
2020-08-21: 0
2020-08-22: 0
2020-08-23: 35.462821769489636
2020-08-24: 59.41867208762467
2020-08-25: 82.37056712171913
2020-08-26: 14.477463916504071
2020-08-27: 4.684671631350909
2020-08-28: 0
2020-08-29: 0
2020-08-30: 10.140008035433283
2020-08-31: 37.27947479522764

Predicting for Bangladesh__nan
2020-08-01: 1101.3534838926716
2020-08-02: 1113.7101495246338
2020-08-03: 688.599134700157
2020-08-04: 665.9745256113409
2020-08-05: 1152.9379657691738
2020-08-06: 1018.8180135788922
2020-08-07: 620.2749286533898
2020-08-08: 841.2700974101906
2020-08-09: 615.4720144149461
2020-08-10: 467.4535372581885
2020-08-11: 432.9180618108881
2020-08-12: 636.3081872356507
2020-08-13: 775.1193300850983
2020-08-14: 396.32852836417004
2020-08-15: 281.8837732796503
2020-08-16: 449.16190542214827
2020-08-17: 245.47465967834916
2020-08-18: 168.8876308871254
2020-08-19: 704.7597340377791
2

2020-08-01: 2294.9466771660714
2020-08-02: 447.8062468451571
2020-08-03: 5.947527008132425
2020-08-04: 0
2020-08-05: 1595.9082219080483
2020-08-06: 1327.3808124247757
2020-08-07: 1260.7030296010616
2020-08-08: 1780.0788827946767
2020-08-09: 0
2020-08-10: 0
2020-08-11: 112.58471814012739
2020-08-12: 941.0976010225348
2020-08-13: 1258.0922821625677
2020-08-14: 1898.4558916691335
2020-08-15: 1018.6096097413744
2020-08-16: 233.79742030678995
2020-08-17: 0
2020-08-18: 0
2020-08-19: 1681.8663463374564
2020-08-20: 1995.2693329089793
2020-08-21: 2065.208355327036
2020-08-22: 1380.9761211077166
2020-08-23: 841.3429733398646
2020-08-24: 0
2020-08-25: 492.62682775815347
2020-08-26: 1812.262595467269
2020-08-27: 2358.973562797615
2020-08-28: 2368.30486260148
2020-08-29: 1519.8687667646586
2020-08-30: 932.6653408188553
2020-08-31: 173.74991897770644

Predicting for Brazil__Ceara
2020-08-01: 564.411543627497
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0
2020-08-05: 418.16008994772267
2020-08-06: 846.585

2020-08-22: 2051.5198891588975
2020-08-23: 919.5568050812745
2020-08-24: 0
2020-08-25: 740.8818468096574
2020-08-26: 3165.0304897087667
2020-08-27: 4690.0482646073115
2020-08-28: 4643.796137150107
2020-08-29: 2221.209999693318
2020-08-30: 952.0588365109246
2020-08-31: 15.39060916399535

Predicting for Brazil__Rio de Janeiro
2020-08-01: 1691.2896829481738
2020-08-02: 0
2020-08-03: 0
2020-08-04: 0
2020-08-05: 53.48087021697734
2020-08-06: 1032.6587064701312
2020-08-07: 1989.5146038850899
2020-08-08: 1764.6324699455517
2020-08-09: 0
2020-08-10: 0
2020-08-11: 0
2020-08-12: 0
2020-08-13: 1234.4514102579656
2020-08-14: 2331.8052015718536
2020-08-15: 1580.6045136272019
2020-08-16: 625.3113191854849
2020-08-17: 0
2020-08-18: 0
2020-08-19: 713.4742981997001
2020-08-20: 1617.1130464888433
2020-08-21: 2478.3298544459963
2020-08-22: 1959.5158789044153
2020-08-23: 989.3896280775207
2020-08-24: 0
2020-08-25: 319.9363211925628
2020-08-26: 953.6420490952812
2020-08-27: 1856.1193853378677
2020-08-28: 2

2020-08-01: 188.50458139010877
2020-08-02: 111.82493254100457
2020-08-03: 0
2020-08-04: 52.37436895518755
2020-08-05: 240.46681512948163
2020-08-06: 73.29332368177393
2020-08-07: 18.226884859487928
2020-08-08: 179.28820291794716
2020-08-09: 148.07302562520982
2020-08-10: 0
2020-08-11: 67.13835054589798
2020-08-12: 227.23570384108214
2020-08-13: 124.19273025128047
2020-08-14: 20.721185548446677
2020-08-15: 187.26176959255707
2020-08-16: 203.96625576830684
2020-08-17: 0
2020-08-18: 114.5291272926219
2020-08-19: 251.1828364824043
2020-08-20: 202.62144893651967
2020-08-21: 76.82661541656384
2020-08-22: 212.05916738561393
2020-08-23: 230.58156516422332
2020-08-24: 77.32430134919089
2020-08-25: 151.62590324288834
2020-08-26: 259.89532190536715
2020-08-27: 240.8640081972266
2020-08-28: 125.54100238483596
2020-08-29: 223.62242493170865
2020-08-30: 250.54215718510997
2020-08-31: 121.71098766268402

Predicting for Central African Republic__nan
2020-08-01: 0
2020-08-02: 0
2020-08-03: 0
2020-08-04

...

2020-08-19: 445.85596788568967
2020-08-20: 632.2237588159695
2020-08-21: 1157.9110694277401
2020-08-22: 1604.3587834956406
2020-08-23: 2300.023900359515
2020-08-24: 1693.341225727795
2020-08-25: 790.4429112509824
2020-08-26: 291.6589168910104
2020-08-27: 583.9418002141714
2020-08-28: 1307.7341663588413
2020-08-29: 1608.360811796023
2020-08-30: 2219.456487041377
2020-08-31: 1687.436616306039

Predicting for United States__Louisiana
2020-08-01: 1511.7852245944032
2020-08-02: 1358.664840900095
2020-08-03: 999.7686859558249
2020-08-04: 228.1563129534747
2020-08-05: 1113.473870353379
2020-08-06: 1503.085491695872
2020-08-07: 1595.1625331429614
2020-08-08: 1485.769620951749
2020-08-09: 450.3660008917587
2020-08-10: 602.2648383333213
2020-08-11: 58.16249110701122
2020-08-12: 775.7200558758842
2020-08-13: 617.5280367161862
2020-08-14: 1440.5744760991465
2020-08-15: 953.9580565678153
2020-08-16: 53.4806084138643
2020-08-17: 142.1845700284079
2020-08-18: 0
2020-08-19: 1011.6983464430044
2020-08-

2020-08-23: 223.4935974481665
2020-08-24: 167.57269476931208
2020-08-25: 0
2020-08-26: 0
2020-08-27: 0
2020-08-28: 123.48728945319404
2020-08-29: 126.974005690753
2020-08-30: 214.9998455450737
2020-08-31: 127.48563970765585

Predicting for United States__North Carolina
2020-08-01: 4976.305073353532
2020-08-02: 5991.557831344028
2020-08-03: 5732.1074105242715
2020-08-04: 4878.805406085229
2020-08-05: 4027.2448262015223
2020-08-06: 2928.873721489135
2020-08-07: 3250.3272567874824
2020-08-08: 5124.708837982523
2020-08-09: 5119.929614666632
2020-08-10: 6231.833329980691
2020-08-11: 4667.132683974891
2020-08-12: 3178.897549992485
2020-08-13: 2507.504371765125
2020-08-14: 2827.732869347356
2020-08-15: 3927.4853408134104
2020-08-16: 5481.5605365774145
2020-08-17: 5625.816977510288
2020-08-18: 4270.663245428693
2020-08-19: 3759.364939331095
2020-08-20: 2313.1843118782203
2020-08-21: 2433.971404442064
2020-08-22: 4123.171194370706
2020-08-23: 5835.688022728155
2020-08-24: 5855.7366058417365
202

2020-08-24: 4671.018531147586
2020-08-25: 4392.40825965829
2020-08-26: 4361.8128541832975
2020-08-27: 5188.150045028289
2020-08-28: 4113.26794005799
2020-08-29: 1910.1147471843892
2020-08-30: 4050.376282490039
2020-08-31: 4516.374598968173

Predicting for United States__Oklahoma
2020-08-01: 1782.872048054097
2020-08-02: 3100.7098853881125
2020-08-03: 4064.9767932018262
2020-08-04: 3059.408331754821
2020-08-05: 1350.4989620609308
2020-08-06: 1298.64774970109
2020-08-07: 958.9856255655584
2020-08-08: 1925.065931671446
2020-08-09: 2944.5867991335126
2020-08-10: 4268.512098513364
2020-08-11: 3056.069505763032
2020-08-12: 1145.783530873442
2020-08-13: 766.6698161416958
2020-08-14: 759.9351934299306
2020-08-15: 1611.6968741958462
2020-08-16: 2991.789507868009
2020-08-17: 4086.5078453855435
2020-08-18: 2836.2696891527917
2020-08-19: 1479.860465741613
2020-08-20: 472.858305528375
2020-08-21: 602.4062176285034
2020-08-22: 1541.2529796044555
2020-08-23: 3236.891797247703
2020-08-24: 4077.8736902

2020-08-29: 1523.1554440926734
2020-08-30: 2277.977760810981
2020-08-31: 1973.1572291235607

Predicting for United States__Virginia
2020-08-01: 1993.4509849186375
2020-08-02: 2433.788010349564
2020-08-03: 2875.115106415309
2020-08-04: 2584.5668962580753
2020-08-05: 2688.6545840638823
2020-08-06: 2162.636063697157
2020-08-07: 1652.8097130906558
2020-08-08: 2162.7941270993047
2020-08-09: 1997.7853676173977
2020-08-10: 2577.76417044014
2020-08-11: 2675.301554107247
2020-08-12: 1828.9063271590335
2020-08-13: 1565.9339211417214
2020-08-14: 1283.7275495796844
2020-08-15: 1117.6141854692087
2020-08-16: 1801.8784686491972
2020-08-17: 2278.0760304333353
2020-08-18: 2156.524743552883
2020-08-19: 2028.3197575083004
2020-08-20: 1488.3433078375706
2020-08-21: 941.0087943586467
2020-08-22: 1131.6875000188538
2020-08-23: 1993.7426876569289
2020-08-24: 2272.4348452085287
2020-08-25: 2363.9565486277647
2020-08-26: 1977.186301003288
2020-08-27: 1328.463163057373
2020-08-28: 960.3666202032495
2020-08-29:

2020-08-01: 6496.047274703903
2020-08-02: 6708.944699403232
2020-08-03: 6447.54702924026
2020-08-04: 4998.727767787101
2020-08-05: 4751.356388896319
2020-08-06: 4546.182931438604
2020-08-07: 4576.318522889795
2020-08-08: 7427.635150242105
2020-08-09: 6965.084564011252
2020-08-10: 6884.2988171838215
2020-08-11: 5784.768784785189
2020-08-12: 4065.8003415304597
2020-08-13: 3841.838214827196
2020-08-14: 4672.385526639867
2020-08-15: 5839.4893868372765
2020-08-16: 7080.144400287321
2020-08-17: 6697.322546440278
2020-08-18: 4882.587438137831
2020-08-19: 4569.3548115277345
2020-08-20: 3726.050938612474
2020-08-21: 4202.430454314473
2020-08-22: 5857.496072841093
2020-08-23: 7666.063547532426
2020-08-24: 6762.798854283226
2020-08-25: 5410.900878662588
2020-08-26: 4538.445171748349
2020-08-27: 3653.505336087444
2020-08-28: 4735.737544308275
2020-08-29: 5936.982725941978
2020-08-30: 7766.912536754831
2020-08-31: 6968.451227554587

Predicting for Zambia__nan
2020-08-01: 35.41710638908934
2020-08-0

In [49]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,0.0
214,Aruba,NaN,2020-08-02,0.0
215,Aruba,NaN,2020-08-03,0.0
216,Aruba,NaN,2020-08-04,0.0
217,Aruba,NaN,2020-08-05,0.0


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [50]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [51]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,0.0
Aruba,,2020-08-02,0.0
Aruba,,2020-08-03,0.0
Aruba,,2020-08-04,0.0
Afghanistan,,2020-08-01,92.87378979947317
Afghanistan,,2020-08-02,1.2652484666498758
Afghanistan,,2020-08-03,72.53020814220258
Afghanistan,,2020-08-04,134.0990297639328
Angola,,2020-08-01,44.22028773532953


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [52]:
import sys,os,os.path
sys.path.append(os.path.expanduser('/home/thinng/code/2020/covid-xprize/'))

In [53]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [54]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [55]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 268 ms, sys: 44.9 ms, total: 313 ms
Wall time: 7.02 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [56]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-24
End date: 2021-06-22


In [57]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [58]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-24 to 2021-06-22...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 5.68 s, sys: 758 ms, total: 6.44 s
Wall time: 2min 27s
